# 0.0 Preprocessing Steps

## Data Augmentation for Pose Diversity
* Problem: Only one good photo available for each pose.
* Solution: Increase data variety by flipping each image horizontally and vertically, creating new orientations of the same pose.

In [30]:
import os
from PIL import Image
from shutil import copyfile

def is_image_file(filename):
    return filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff'))

def flip_image_horizontally(image_path, save_path):
    img = Image.open(image_path)
    flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
    flipped_img.save(save_path)

def flip_image_vertically(image_path, save_path):
    with Image.open(image_path) as img:
        flipped_img = img.transpose(Image.FLIP_TOP_BOTTOM)
        flipped_img.save(save_path)

def diversity_datasets(root_dir, inv=True):
    error = 0
    copy_count = 0
    flipped_horizontal_count = 0
    flipped_vertical_count = 0

    lft_images_dir = root_dir + "lft/"
    rgt_images_dir = root_dir + "rgt/"

    for dir_path, suffix in [(lft_images_dir, '-lft.png'), (rgt_images_dir, '-rgt.png')]:
        for filename in os.listdir(dir_path):
            if is_image_file(filename):
                base_filename = os.path.splitext(filename)[0]
                source_path = os.path.join(dir_path, filename)
                save_path = os.path.join(root_dir, base_filename + suffix)
                copyfile(source_path, save_path)
                copy_count += 1
    print(f"Successfully copied {copy_count} images")

    for filename in os.listdir(root_dir):
        if is_image_file(filename) and filename.lower().endswith('-lft.png'):
            base_filename = filename.replace('-lft.png', '')
            flipped_path = os.path.join(root_dir, base_filename + '-rgt.png')
            source_path = os.path.join(root_dir, filename)
            flip_image_horizontally(source_path, flipped_path)
            flipped_horizontal_count += 1 
        elif is_image_file(filename) and filename.lower().endswith('-rgt.png'):
            base_filename = filename.replace('-rgt.png', '')
            flipped_path = os.path.join(root_dir, base_filename + '-lft.png')
            source_path = os.path.join(root_dir, filename)
            flip_image_horizontally(source_path, flipped_path)
            flipped_horizontal_count += 1 
        else:
            error += 1
            
    print(f"Successfully flipped horizontally {flipped_horizontal_count} images")

    if inv:
        for filename in os.listdir(root_dir):
            if is_image_file(filename):
                source_path = os.path.join(root_dir, filename)
                save_path = os.path.join(root_dir, os.path.splitext(filename)[0] + '-inv.png')
                flip_image_vertically(source_path, save_path)
                flipped_vertical_count += 1
            print(f"Successfully flipped vertically {flipped_vertical_count} images")
    else:
        print(f"Vertical flipping skipped")

In [22]:
root_dir = 'data/external/positions/grip/'
results = diversity_datasets(root_dir, True)

Successfully copied 39 images
Successfully flipped horizontally 39 images
Successfully flipped vertically 78 images


In [31]:
root_dir = 'data/external/tricks/beginner/'
results = diversity_datasets(root_dir, False)

Successfully copied 106 images
Successfully flipped horizontally 106 images
Vertical flipping skipped


In [32]:
root_dir = 'data/external/tricks/intermediate/'
results = diversity_datasets(root_dir, False)

Successfully copied 89 images
Successfully flipped horizontally 89 images
Vertical flipping skipped


In [33]:
root_dir = 'data/external/tricks/advanced/'
results = diversity_datasets(root_dir, False)

Successfully copied 83 images
Successfully flipped horizontally 83 images
Vertical flipping skipped
